<a href="https://colab.research.google.com/github/ThantiwaChaimanee/AI-Intern/blob/Wa/pic_label_to_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gspread

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import os
import re
import shutil

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


mypath คือ โฟลเดอร์ต้นทาง
new_path คือ โฟลเดอร์ปลาย( สร้างขึ้นมา แล้ว copy path มาใส่)

In [ ]:
from os import listdir
from os.path import isfile, join

mypath = '/content/drive/MyDrive/INtern/Project/carMatching/carMatchingFullModel/yolov5/runs/detect/exp13_isuzu'
new_path = '/content/drive/MyDrive/INtern/Project/carMatching/carMatchingFullModel/result_csv/exp13'

pic_list = [f for f in listdir(mypath) if isfile(join(mypath, f))]


In [ ]:
front_name = lambda f_name: f_name.split('_')[0]

In [ ]:
name_pic_folder = sorted(list(set([front_name(f) for f in pic_list])))

In [ ]:
label_path = f'{mypath}/labels'
label_list = [f for f in listdir(label_path) if isfile(join(label_path, f))]


In [ ]:
for f in name_pic_folder:
  os.mkdir(f'{new_path}/{f}')
  os.mkdir(f'{new_path}/{f}/labels')

In [ ]:
for pic in pic_list:
  shutil.copyfile(f'{mypath}/{pic}' , f'{new_path}/{front_name(pic)}/{pic}')

In [ ]:
for lab in label_list:
  shutil.copyfile(f'{label_path}/{lab}' , f'{new_path}/{front_name(lab)}/labels/{lab}')

In [ ]:
def read_label(file):
  with open(file, "r") as f:
    contents = f.readlines()
    length = len(contents)
    pl = []
    if length >= 2:
      for line in range(length):
        pl.append(contents[line][0])
      if len(set(pl)) == 1:
        return contents[line][0]
      else:
        return '-'
    else:
      return contents[0][0]

# def read_label(file):
#   with open(file) as f:
#     contents = f.read()
    
#   return contents[0]

In [ ]:

predeict_dict = {'-':'Not detect', 
                   '0':'Back',
                   '1':'Front',
                   '2':'Left',
                   '3':'Right'
                   }

pic_name = []
pic_label =[]
predict_name = []
pic_result = []
pic_remark = []


for folder in listdir(new_path):
  name_label_file_list = [f for f in listdir(f'{new_path}/{folder}/labels') if isfile(join(label_path, f))]



  pic_file_list = [f.split('.')[0] for f in listdir(f'{new_path}/{folder}') if isfile(join(f'{new_path }/{folder}', f))]

  pic_name.extend(pic_file_list)
  
  
  temp_lab = ''

  for pic in pic_file_list:
   
    if pic in [name.split('.')[0] for name in name_label_file_list]:

      
      lab = f'{pic}.txt'

      lab_num = read_label(f'{new_path}/{folder}/labels/{lab}')

    else :
      lab_num = '-'


    pic_label.append(lab_num)
    predict_name.append(predeict_dict[lab_num])
    temp_lab += f'{lab_num}'


  
  for pic in pic_file_list:
    if set('0123').issubset(set(temp_lab)):
      pic_result.append('Success')
      pic_remark.append('-')
    else:
      pic_result.append('Not Success')

      rem =  sorted(list(set('0123') - set(temp_lab)))    
      rem = [predeict_dict[r] for r in rem ]
      rem = ",".join(rem)

      pic_remark.append(rem)





pic_fol = list(map(lambda x:front_name(x), pic_name))



In [ ]:
df = pd.DataFrame(list(zip(pic_fol, pic_name,pic_label,predict_name,pic_result,pic_remark)),
               columns =['folder_name', 'image_name','predict_number','predeict_name','result','remark'])

df

,folder_name,image_name,predict_number,predeict_name,result,remark
0,65001000120,65001000120_001,0,Back,Success,-
1,65001000120,65001000120_002,3,Right,Success,-
2,65001000120,65001000120_003,1,Front,Success,-
3,65001000120,65001000120_004,2,Left,Success,-
4,65001000120,65001000120_005,-,Not detect,Success,-
...,...,...,...,...,...,...
317,65001E00737,65001E00737_001,-,Not detect,Not Success,"Back,Left,Right"
318,65001E00737,65001E00737_002,-,Not detect,Not Success,"Back,Left,Right"
319,65001E00737,65001E00737_003,-,Not detect,Not Success,"Back,Left,Right"
320,65001E00737,65001E00737_004,1,Front,Not Success,"Back,Left,Right"


In [ ]:
changed1 = set()
for fn in df.image_name:
  #print(changed1)
  if front_name(fn) not in changed1:
    changed1.add(front_name(fn))
  else:
    df.loc[(df['image_name'] == fn)           ,['result','remark','folder_name']] = '-'  

In [ ]:
df

,folder_name,image_name,predict_number,predeict_name,result,remark
0,65001000120,65001000120_001,0,Back,Success,-
1,-,65001000120_002,3,Right,-,-
2,-,65001000120_003,1,Front,-,-
3,-,65001000120_004,2,Left,-,-
4,-,65001000120_005,-,Not detect,-,-
...,...,...,...,...,...,...
317,65001E00737,65001E00737_001,-,Not detect,Not Success,"Back,Left,Right"
318,-,65001E00737_002,-,Not detect,-,-
319,-,65001E00737_003,-,Not detect,-,-
320,-,65001E00737_004,1,Front,-,-


In [ ]:
path = '/content/drive/MyDrive/INtern/Project/carMatching/carMatchingFullModel/result_csv/exp13/exp13_isuzu.csv'

with open(path, 'w', encoding = 'utf-8') as result:
    df.to_csv(result)
# df.to_csv('result_exp27.csv', index=False)